In [ ]:
%matplotlib inline

In [ ]:
# standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# will use biophython to deal with biological sequences
from Bio import SeqIO, Seq
from Bio.Alphabet import IUPAC

# and of course the Ruggero-detector
import ruggero_detector as rd

# Keras imports
from keras.layers import multiply, Input, Dense
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
import keras.backend as K
from keras.preprocessing import sequence

# our beloved fancy progress bar
from tqdm import tqdm_notebook as tqdm

# 2019-02-18 What makes a promoter?
Using the technology that we developed for the Ruggero-detector, we can try to ask the question: what makes a promoter a promoter? We can feed sequences of DNA to a Ruggero-detector, and then try to use the attention mechanism that we developed earlier to figure out what makes the network shifts its prediction on the sequence.

## Getting promoter data

### SuRE method

The publication by J. van Arensbergen et al (Nature Biotechnology 2017) gives a genome-wide map of positions and their promoter activity.

The data file is about 6 Gb in size. It would be unnecessary to load all the data in memory. That's why I wrote a simple C program that runs very fast, which can do the job neatly. We need to make sure that we get random lines from that file, taking care of the fact that we uniformly select lines from promoters and non-promoters.

In [ ]:
# parse data file
fname = '../code/randlines/test.dat'
data = pd.read_csv(fname, sep='\t')
chromosomes = data['chr'].unique()

Now we need to go and fetch the sequences that correspond to the positions in the data set. We will use Biopython to load the human genome and read the information on the sequence.

In [ ]:
# load human genome
genome_file = '/mnt/shared/seq/hg19/hg19_unmasked.fasta'
genome = SeqIO.index(hg38_genome_file,'fasta', alphabet=IUPAC.unambiguous_dna)

Now, Biopython is fantastic for extracting sequences of DNA, but it is much less fantastic when it comes to speed. If we want to have the hope to get somewhere, we need to get the data ordered by chromosome, otherwise it's going to take forever.

In [ ]:
# build a dictionary of data points, which correspond to a chromosome
data_chr = {chromosome : data.loc[data['chr'] == chromosome] for chromosome in chromosomes}

We can now use an iteration over all the chromosomes and fetch the information.

In [ ]:
# first, I prepare the dictionary that encodes the sequence alphabet
seq_to_n = {'A' : 0, 'T' : 1, 'C' : 2, 'G' : 3, 'N' : 4}
n_to_seq = {v : k for k, v in seq_to_n.items()}

In [ ]:
sequences = []
targets = []
for chromosome in chromosomes :
    
    print("Parsing data of chromosome " + chromosome)

    # get the reads corresponding to the current chromosome
    chromosome_data = data_chr[chromosome]
    c = genome[chromosome]
    
    # iterate through the reads
    for i, row in enumerate(chromosome_data.iterrows()) :
        
        # parse the information in the row
        r = row[1]
        start, end, strand = r['start'], r['end'], r['strand']
        myseq = c.seq[start:end]
        if strand == '-' :
            myseq = myseq.reverse_complement()
        myseq = myseq.upper()
        
        # let's figure out whether the sequence was classified as promoter or not
        counts = sum(row[1][5:11])
        if counts < 10 :
            promoter = 0
        else :
            promoter = 1
        
        # encode the sequence in the format that will be comprehensible by the AI
        seq_encoded = rd.encode_sentence(myseq, seq_to_n)
        sequences.append(seq_encoded)
        targets.append(promoter)

Okay, we're ready to prepare the training and testing data sets.

In [ ]:
N = len(sequences)
ntrain = int(0.9 * N)
ntest = N - ntrain
padded_sequences = sequence.pad_sequences(sequences, maxlen=200)
train_data = to_categorical(padded_sequences[:ntrain])
train_targets = targets[:ntrain]
test_data = to_categorical(padded_sequences[ntrain:])
test_targets = targets[ntrain:]

Okay, we're ready to fire our LSTM and see what happens.

In [ ]:
model = Sequential()

# LSTM layer
lstm_units = 128
model.add(LSTM(lstm_units, return_sequences=False, input_shape=(None, len(seq_to_n))))

# output layer
model.add(Dense(1, activation='softmax'))

# compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# define checkpointer and fit the model
checkpointer = ModelCheckpoint(filepath='../data/promoter-detector.hdf5', 
                               verbose=1, save_best_only=True)
model.fit(train_data, train_targets,
          batch_size=32,
          epochs=10,
          validation_split = 0.2,
          callbacks=[checkpointer])

Okay, no matter what I try here doesn't give any meaningful results.

I think I'd better think of a different strategy when it comes to trying to identify promoter sequences: the fact that I padded the sequences to feed them to the LSTM is not the greatest idea. I might be overlooking entirely the important bits of information from the sequences.

## Different sequence management

I put the bit of code that extracts the random sequences in a script that works in a two-step way: first step is to extract the random sequences (`randlines`) and the second script fetches the sequences from the genome.

In [ ]:
in_fname = '../data/rand_sequences-SuRE.dat'
def parse_sequence_file(fname) :
    sequences = []
    targets = []
    with open(in_fname, 'r') as f :
        for line in f :
            target, sequence = line.strip('\n').split(' ')
            sequences.append(sequence)
            targets.append(int(target))
    return sequences, targets
sequences, targets = parse_sequence_file(in_fname)

Let's have a look at the distribution of lengths of the sequences.

In [ ]:
seq_lengths = np.array([len(s) for s in sequences])
plt.hist(seq_lengths, bins = 100)
plt.show()

So from here it's clear that we can actually train a lot of times on sequences that have a lot of data points, located in that bulk between sequence length 500-1500.

In [ ]:
sequences_encoded = []
for sequence in sequences :
    seq_encoded = rd.encode_sentence(sequence, alpha_to_n=seq_to_n)
    seq_encoded = to_categorical(seq_encoded, num_classes=len(seq_to_n))
    sequences_encoded.append(seq_encoded)

In [ ]:
def train_generator(sequences_encoded, targets, alphabet_size) :
    
    # get the value of the sequence lengths in an array
    seq_lengths = np.array([s.shape[0] for s in sequences_encoded])
    max_length = seq_lengths.max()
    
    # we also set a minimum length of sequence that we want to pass to the
    # training
    min_length = 100
    
    # the generator loops forever
    seq_length = min_length
    while True :
        
        # this statement allows to loop indefinitely through the same data
        if seq_length==max_length :
            seq_length = min_length
        
        # select all the sequences that have that fixed length, provided that there
        # are a sufficient amount of data points to take into consideration
        seq_mask = seq_lengths==seq_length
        nseqs = sum(seq_mask)
        if nseqs < 10 :
            seq_length += 1
            continue

        # init the objects that will make the output
        idxs = np.where(seq_mask)[0]
        x_train = np.zeros((nseqs, seq_length, alphabet_size))
        y_train = []
        for i, idx in enumerate(idxs) :
            seq = sequences[idx]
            x_train[i, :, :] = sequences_encoded[idx]
            y_train.append(targets[idx])
        
        # increment the sequence length
        seq_length += 1
        
        # and finally, yield
        yield x_train, y_train

In [ ]:
model = Sequential()

# LSTM layer
lstm_units = 32
model.add(LSTM(lstm_units, return_sequences=False, input_shape=(None, len(seq_to_n))))

# output layer
model.add(Dense(1, activation='softmax'))

# compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# define checkpointer and fit the model
checkpointer = ModelCheckpoint(filepath='../data/promoter-detector.hdf5', 
                               verbose=1, save_best_only=True)
model.fit_generator(train_generator(sequences_encoded, targets, seq_to_n),
          steps_per_epoch = 100,
          epochs=10,
          callbacks=[checkpointer])

It won't train. Let's try with dinucleotides or trinucleotides.

## More words

I want to try here to see what happens if we include more words in the alphabet that we are feeding to the model. I'll dig back my functions that I wrote when I was working on the AI to distinguish between genomes.

It turns out that it's very slow for python to process the strings and convert them to their encoding, so I wrote a small C program that does the job very rapidly.

In [ ]:
def sequence_decode(num, mapping, base=5, length=None) :
    N = num
    i = 0
    seq = ''
    while N>0 :
        ai = N%base**(i+1)//base**i
        N -= ai*base**i
        i+=1
        seq += mapping[ai]
    if length is not None :
        seq += mapping[0]*(length-len(seq))
    return seq[::-1]

In [ ]:
def sequence_encode(sequence, mapping, base=5) :
    l = len(sequence)
    return np.sum([base**(l-i-1)*mapping[sequence[i]] for i in range(l)])

In [ ]:
sequences_encoded_fname = '../data/rand_sequences-encoded-SuRE.dat'
sequences_encoded_cat = []
nletters = 3
base = len(seq_to_n)
alphabet_size = base**nletters
with open(sequences_encoded_fname, 'r') as f :
    for line in f :
        sequence_encoded = list(map(int, line.split()))
        sequences_encoded_cat.append(to_categorical(sequence_encoded, num_classes=alphabet_size))

In [ ]:
sequences_encoded_cat[0].shape

In [ ]:
model = Sequential()

# LSTM layer
lstm_units = 32
model.add(LSTM(lstm_units, return_sequences=False, input_shape=(None, len(seq_to_n))))

# output layer
model.add(Dense(1, activation='softmax'))

# compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# define checkpointer and fit the model
checkpointer = ModelCheckpoint(filepath='../data/promoter-detector.hdf5', 
                               verbose=1, save_best_only=True)
model.fit_generator(train_generator(sequences_encoded_cat, targets, alphabet_size),
          steps_per_epoch = 100,
          epochs=10,
          callbacks=[checkpointer])